# Set of general analytical queries

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from google.cloud import bigquery, storage

In [ ]:
bq = bigquery.Client()

## Defining some BQ queries

In [ ]:
last_save_time_query = """
    select distinct(user_id), TIMESTAMP_MILLIS(max(time_transaction_occurred)) as save_time
    from ops.user_behaviour where transaction_type = 'SAVING_EVENT'
    group by user_id
"""

In [ ]:
boost_redemption_query = """
    with boost_offers as (
            select *, TIMESTAMP_MILLIS(created_at) as creation_timestamp 
            from ops.all_user_events 
            where event_type like 'BOOST_CREATED%'

    ), boost_redemptions as (
            select *, TIMESTAMP_MILLIS(created_at) as creation_timestamp 
            from ops.all_user_events 
            where event_type = 'BOOST_REDEEMED'
    )
    select boost_offers.user_id, boost_offers.event_type, boost_offers.context, 
        boost_offers.creation_timestamp as boost_creation_time, boost_redemptions.creation_timestamp as boost_redemption_time,  
        TIMESTAMP_DIFF(boost_redemptions.creation_timestamp, boost_offers.creation_timestamp, HOUR) as time_from_boost_to_save
    from boost_offers left join boost_redemptions on boost_offers.user_id = boost_redemptions.user_id
        where 
        TIMESTAMP_DIFF(boost_redemptions.creation_timestamp, boost_offers.creation_timestamp, HOUR) is null
    """

In [ ]:
cohort_behaviour_query = """
    select user_id, min(TIMESTAMP_MILLIS(time_transaction_occurred)), sum(amount) from ops.user_behaviour group by user_id
"""

In [ ]:
saves_per_month = """
    select EXTRACT(MONTH from TIMESTAMP_MILLIS(time_transaction_occurred)) as save_month, unit, sum(amount) as sum, avg(amount), count(*) as count from ops.user_behaviour 
    where transaction_type = 'SAVING_EVENT' 
    group by save_month, unit 
    order by save_month desc;
"""

In [ ]:
withdrawals_with_next_save = """
    with withdrawal_events as (
      select user_id, event_type, timestamp_millis(time_transaction_occurred) as withdrawal_time, context
      from ops.all_user_events where event_type = 'ADMIN_SETTLED_WITHDRAWAL'
    ),
    save_events as (
      select user_id, transaction_type, timestamp_millis(time_transaction_occurred) as save_time, amount, unit
      from ops.user_behaviour where transaction_type = 'SAVING_EVENT'
    )
    select withdrawal_events.user_id, withdrawal_events.withdrawal_time as withdrawal_time, 
    min(save_events.save_time) as next_save_time, count(save_events.save_time > withdrawal_time) as subsequent_save_count,
    from withdrawal_events left join save_events on withdrawal_events.user_id = save_events.user_id
    where (save_events.save_time > withdrawal_time)
    group by user_id, withdrawal_events.withdrawal_time
"""

In [ ]:
withdrawals_any_save = """
    with withdrawal_events as (
      select user_id, event_type, timestamp_millis(time_transaction_occurred) as withdrawal_time, context
      from ops.all_user_events where event_type = 'ADMIN_SETTLED_WITHDRAWAL'
    ),
    save_events as (
      select user_id, transaction_type, timestamp_millis(time_transaction_occurred) as save_time, amount, unit
      from ops.user_behaviour where transaction_type = 'SAVING_EVENT'
    )
    select count(distinct(withdrawal_events.user_id)),
    from withdrawal_events inner join save_events on withdrawal_events.user_id = save_events.user_id
    where (save_events.save_time > withdrawal_time)
"""

In [ ]:
acc_open_to_withdrawal = """
with account_open_events as (
  select user_id, event_type, timestamp_millis(time_transaction_occurred) as account_open_time, context
  from ops.all_user_events where event_type = 'USER_CREATED_ACCOUNT'
),
withdrawal_events as (
  select user_id, event_type, timestamp_millis(time_transaction_occurred) as withdrawal_time, context
  from ops.all_user_events where event_type = 'ADMIN_SETTLED_WITHDRAWAL'
)
select account_open_events.user_id, account_open_events.account_open_time as account_open_time, 
min(withdrawal_events.withdrawal_time) as first_withdraw_time,
TIMESTAMP_DIFF(min(withdrawal_events.withdrawal_time), account_open_time, DAY) as days_to_withdraw,
from account_open_events inner join withdrawal_events on account_open_events.user_id = withdrawal_events.user_id
group by user_id, account_open_time;
"""

In [ ]:
last_save_time_df = bq.query(last_save_time_query).to_dataframe()

In [ ]:
current_time = pd.Timestamp.now(tz='UTC')
last_save_time_df['days_since'] = (current_time - last_save_time_df['save_time']).dt.days

In [ ]:
day_divisions = [0, 15, 30, 45, 60, 75, 90, 105, 120]
last_save_time_df.hist(bins=day_divisions)

In [ ]:
pd.cut(last_save_time_df['days_since'], [0, 30, 60, 90, 120]).value_counts().sort_index()

In [ ]:
acc_withdraw_df = bq.query(acc_open_to_withdrawal).to_dataframe()

In [ ]:
acc_withdraw_df["days_to_withdraw"].describe()

In [ ]:
message_app_open = """
    with message_sent_event as (
      select user_id, event_type, timestamp_millis(time_transaction_occurred) as time_message_sent, timestamp_trunc(timestamp_millis(time_transaction_occurred), HOUR) as message_hour
      from ops.all_user_events where event_type = 'MESSAGE_SENT'
    ),
    app_open_event as (
      select user_id, event_type, timestamp_millis(time_transaction_occurred) as app_open_time
      from ops.all_user_events where event_type = 'USER_OPENED_APP'
    )
    select 
      message_sent_event.user_id, message_sent_event.message_hour, min(app_open_event.app_open_time) as earliest_app_open,
    from 
      message_sent_event left join app_open_event on (message_sent_event.user_id = app_open_event.user_id and message_hour < app_open_time)
    where 
      message_hour < timestamp_sub(current_timestamp, interval 2 DAY) and (
        app_open_time is null or timestamp_diff(app_open_time, message_hour, HOUR) < 48
      )
    group by user_id, message_hour
"""

In [ ]:
msg_open_df = bq.query(message_app_open).to_dataframe()

In [ ]:
msg_open_df.dtypes

In [ ]:
msg_open_df.index = msg_open_df['message_hour']

In [ ]:
monthly_counts = msg_open_df.resample('M').count()
monthly_counts.head()

In [ ]:
monthly_counts['open_rate'] = monthly_counts['earliest_app_open'] / monthly_counts['user_id']

In [ ]:
monthly_counts

In [ ]:
saves_by_cohorts = """
with account_open_events as (
  select user_id, event_type, timestamp_millis(time_transaction_occurred) as account_open_time, context
  from ops.all_user_events where event_type = 'USER_CREATED_ACCOUNT'
),
save_events as (
  select user_id, timestamp_millis(time_transaction_occurred) as save_time, amount / 10000 as save_amount_fc
  from ops.user_behaviour where transaction_type = 'SAVING_EVENT'
)
select account_open_events.user_id, account_open_events.account_open_time, 
count(save_time) as count_save_events, max(save_amount_fc) as max_save, sum(save_amount_fc) as total_save, avg(save_amount_fc) as average_save,
from 
  account_open_events inner join save_events on account_open_events.user_id = save_events.user_id
group by user_id, account_open_time;
"""

In [ ]:
save_df = bq.query(saves_by_cohorts).to_dataframe()

In [ ]:
save_df.dtypes

In [ ]:
save_df.index = save_df['account_open_time']

In [ ]:
save_monthly_cohorts = save_df.resample('M')

In [ ]:
cohort_data = [save_monthly_cohorts['total_save'].sum(), save_monthly_cohorts['average_save'].mean()]

In [ ]:
cohort_data

In [ ]:
boost_inducing = """
with boost_offers as (
        select *, TIMESTAMP_MILLIS(created_at) as boost_creation_time 
        from ops.all_user_events 
        where event_type like 'BOOST_CREATED%'

), save_events as (
        select *, TIMESTAMP_MILLIS(created_at) as next_save_time 
        from ops.all_user_events 
        where event_type = 'SAVING_PAYMENT_SUCCESSFUL'
)
select boost_offers.user_id, boost_creation_time, min(next_save_time) as earliest_save_time
from boost_offers left join save_events 
on boost_offers.user_id = save_events.user_id and boost_creation_time < next_save_time
where boost_creation_time < timestamp_sub(current_timestamp, interval 2 DAY) and (
  next_save_time is null or timestamp_diff(next_save_time, boost_creation_time, HOUR) < 96
)
group by boost_offers.user_id, boost_creation_time
"""

In [ ]:
boost_offer_df = bq.query(boost_inducing).to_dataframe()

In [ ]:
boost_offer_df.dtypes

In [ ]:
boost_offer_df.index = boost_offer_df['boost_creation_time']

In [ ]:
boost_monthly_counts = boost_offer_df.resample('M').count()

In [ ]:
boost_monthly_counts.head()

In [ ]:
boost_monthly_counts['induce_rate'] = boost_monthly_counts['earliest_save_time'] / boost_monthly_counts['boost_creation_time']

In [ ]:
boost_monthly_counts.head()

In [ ]:
boost_offer_df['month'] = boost_offer_df['boost_creation_time'].dt.month

In [ ]:
boost_monthly_agg = boost_offer_df.groupby('month').count()
boost_monthly_agg['unique_users'] = boost_offer_df.groupby('month')['user_id'].nunique()
boost_monthly_agg['boost_per_user'] = boost_monthly_agg['boost_creation_time'] / boost_monthly_agg['unique_users']
boost_monthly_agg['induce_rate'] = boost_monthly_agg['earliest_save_time'] / boost_monthly_agg['unique_users']
boost_monthly_agg = boost_monthly_agg[['boost_creation_time', 'earliest_save_time', 'unique_users', 'boost_per_user', 'induce_rate']]

In [ ]:
boost_monthly_agg

In [ ]:
saves_by_cohorts_after_14_days = """
with first_save_event as (
  select user_id, min(timestamp_millis(time_transaction_occurred)) as first_save_time,
  from ops.all_user_events where event_type = 'SAVING_PAYMENT_SUCCESSFUL' group by user_id
),
save_events as (
  select user_id, timestamp_millis(time_transaction_occurred) as save_time, amount / 10000 as save_amount_fc
  from ops.user_behaviour where transaction_type = 'SAVING_EVENT'
)
select first_save_event.user_id, first_save_time, count(save_time) as later_saves, avg(save_amount_fc) as average_save,
from first_save_event inner join save_events on 
  first_save_event.user_id = save_events.user_id
  and timestamp_diff(save_time, first_save_time, DAY) > 14
where first_save_time < timestamp_sub(current_timestamp, interval 30 DAY)
group by first_save_event.user_id, first_save_time; 
"""

In [ ]:
saves_by_first_save = bq.query(saves_by_cohorts_after_14_days).to_dataframe()

In [ ]:
saves_by_first_save['month'] = saves_by_first_save.first_save_time.dt.month
saves_by_first_save['year'] = saves_by_first_save.first_save_time.dt.year

In [ ]:
saves_by_first_save

In [ ]:
save_stats_by_cohort = saves_by_first_save.groupby(['month', 'year'])

In [ ]:
save_stats_by_cohort.aggregate(
    { 'user_id': pd.Series.nunique, 'average_save': np.mean, 'later_saves': np.mean }
)

In [ ]:
wns_df = bq.query(withdrawals_with_next_save).to_dataframe()

In [ ]:
len(wns_df)